In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf


# GPU 디바이스 목록 가져오기

gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    print("사용 가능한 GPU 디바이스:")
    for gpu in gpus:
        print("디바이스 이름:", gpu.name)
else:
    print("사용 가능한 GPU 디바이스가 없습니다.")

In [ ]:
# 이미지 전처리 코드

# image_dir = "C:\\Users\\crazy\\Downloads\\archive\\img_align_celeba\\img_align_celeba"
# image_dir = "C:\\Users\\crazy\\Downloads\\archive\\img_align_celeba\\10000"
# image_dir = "C:\\Users\\crazy\\Downloads\\archive\\img_align_celeba\\10"
# image_dir = "C:\\Myfolder\\AI_tools\\dfl\\df_NVIDIA_RTX3000_series\\workspace\\data_src\\aligned"
# image_dir = "C:\\Myfolder\\AI_tools\\sd.webui\\this\\fun\\image"
# image_dir = "C:\\Users\\crazy\Downloads\\archive\\img_align_celeba\\facesetav"
image_dir = "C:\\Users\\crazy\Downloads\\archive\\img_align_celeba\\picted"
# crop_width, crop_height = 1024, 1024  # 자를 크기
resize_width, resize_height = 128, 128  # 변경할 크기

# 이미지 목록 가져오기
image_files = os.listdir(image_dir)

# 이미지를 NumPy 배열로 로드하고 크기 변경하기
images = []
for filename in image_files:
    img = tf.keras.preprocessing.image.load_img(os.path.join(image_dir, filename), target_size=(resize_width, resize_height))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0  # 이미지 정규화
    images.append(img)
images = np.array(images)


# 이미지 중에서 무작위로 하나 선택
random_index = random.randint(0, len(images) - 1)
selected_image = images[random_index]

In [ ]:
# # 선택한 이미지 표시
# plt.imshow(selected_image)
# plt.show()

In [ ]:
keras = tf.keras
K = tf.keras.backend

class Sampling(keras.layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
#인코더 부분
encoder_input = keras.layers.Input(shape=(resize_width, resize_height, 3), name="encoder_input")
x = keras.layers.Conv2D(32, (3, 3), strides=2, activation="relu", padding="same")(encoder_input)
x = keras.layers.Conv2D(64, (3, 3), strides=2, activation="relu", padding="same")(x)
x = keras.layers.Conv2D(128, (3, 3), strides=2, activation="relu", padding="same")(x)
shape_before_flattening = K.int_shape(x)[1:]
x = keras.layers.Flatten()(x)
z_mean = keras.layers.Dense(200, name="z_mean")(x)
z_log_var = keras.layers.Dense(200, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.models.Model(encoder_input, [z_mean, z_log_var, z], name="encoder")

#디코더 부분
decoder_input = keras.layers.Input(shape=(200,), name="decoder_input")
x = keras.layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = keras.layers.Reshape(shape_before_flattening)(x)
x = keras.layers.Conv2DTranspose(128, (3, 3), strides=2, activation = 'relu', padding="same")(x)
x = keras.layers.Conv2DTranspose(64, (3, 3), strides=2, activation = 'relu', padding="same")(x)
x = keras.layers.Conv2DTranspose(32, (3, 3), strides=2, activation = 'relu', padding="same")(x)
decoder_output = keras.layers.Conv2D(3,(3, 3),strides = 1, activation="relu", padding="same", name="decoder_output")(x)
decoder = keras.models.Model(decoder_input, decoder_output, name="decoder")

encoder.summary()
decoder.summary()

In [ ]:
class VAE(keras.models.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

        #loss 값을 모니터링 하는 요소
        self.total_loss_tracker = keras.metrics.Mean(name='total_loss')
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def call(self, input):
        z_mean, z_log_var, z = encoder(input)
        reconstruction = decoder(z)
        return z_mean, z_log_var, reconstruction
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, reconstruction = self(data)
            reconstruction_loss = tf.reduce_mean(500 * tf.losses.binary_crossentropy(data, reconstruction, axis=(1,2,3)))
            kl_loss = tf.reduce_mean(tf.reduce_sum(-0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)), axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {m.name: m.result() for m in self.metrics}

In [ ]:
#모델 새로 정의할때
vae = VAE(encoder, decoder)

In [ ]:
vae.compile(optimizer="adam")
vae.fit(images, epochs=200000, batch_size=32)

In [ ]:
# 이미지 중에서 무작위로 하나 선택
random_index1 = random.randint(0, len(images) - 1)
selected_image1 = images[random_index1]

# 또 다른 이미지를 무작위로 선택
random_index2 = random.randint(0, len(images) - 1)
selected_image2 = images[random_index2]

# 선택한 이미지 표시
plt.figure(figsize=(10, 5))  # 그림 크기 조정
plt.subplot(1, 2, 1)  # 1x2 그리드의 첫 번째 위치
plt.imshow(selected_image1)
plt.title("Original Image 1")

# 인코더에 넣어서 임베딩 벡터 확인
embedded1 = np.array(encoder.predict(np.array([selected_image1])))[2]

# 임베딩 벡터를 디코더에 넣어서 결과값 예측
predicted_img1 = decoder.predict(embedded1)
plt.subplot(1, 2, 2)  # 1x2 그리드의 두 번째 위치
plt.imshow(predicted_img1.reshape(resize_width, resize_height, 3))
plt.title("Reconstructed Image 1")

plt.show()

# 선택한 다른 이미지 표시
plt.figure(figsize=(10, 5))  # 그림 크기 조정
plt.subplot(1, 2, 1)  # 1x2 그리드의 첫 번째 위치
plt.imshow(selected_image2)
plt.title("Original Image 2")

# 인코더에 넣어서 임베딩 벡터 확인
embedded2 = np.array(encoder.predict(np.array([selected_image2])))[2]

# 임베딩 벡터를 디코더에 넣어서 결과값 예측
predicted_img2 = decoder.predict(embedded2)
plt.subplot(1, 2, 2)  # 1x2 그리드의 두 번째 위치
plt.imshow(predicted_img2.reshape(resize_width, resize_height, 3))
plt.title("Reconstructed Image 2")

plt.show()

# 두 이미지의 임베딩 벡터의 중간값을 이용해 결과값 예측
predicted_img_combined = decoder.predict((embedded1 + embedded2) / 2)
plt.figure(figsize=(5, 5))
plt.imshow(predicted_img_combined.reshape(resize_width, resize_height, 3))
plt.title("Reconstructed Combined Image")

plt.show()

In [ ]:
# vae.save_weights('vae_weight_celebA.h5')
# vae.load_weights('vae_weight_celebA.h5')

vae.save_weights('vae_sweight_av.h5')
# vae.load_weights('vae_sweight_av.h5')